In [1]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
import re
from bs4 import BeautifulSoup

In [19]:
url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
#url = 'https://web.archive.org/web/20200720171507/https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
# TODO - Wayback machine adds two extra tables at top we need to skip past
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [20]:
tables = soup.find_all('table')
#[len(a) for a in a[0].find_all('tr')]

In [21]:
tmp = re.split('/| ', tables[0].find_all('caption')[0].get_text())
reporting_date = '-'.join(tmp[0:3])
print(reporting_date)

11-16-2020


In [22]:
# Create list of facilities
rows = tables[2].find_all('tr')
# Get List of fields from Header Row
fieldList = []
fields = rows[0].find_all('th')
for field in fields:
    fieldList.append(field.get_text())
#assert fieldList == ['FACILITY NAME', 'COUNTY','TYPE', 'NEW RESIDENT CASES', 'TOTAL RESIDENT CASES', 'NEW RESIDENT DEATHS', 'TOTAL RESIDENT DEATHS',
#                     'NEW STAFF CASES', 'TOTAL STAFF CASES', 'NEW STAFF DEATHS', 'TOTAL STAFF DEATHS']
num_facilities = 0
facilities = []
first_row = True
for row in rows:
    if first_row:
        first_row = False
        continue
    facility = {}
    tds = row.find_all('td')
    num_reported_vals = 0
    facility['ReportingDate'] = reporting_date
    for i in range(len(tds)):
        val = tds[i].get_text()
        if val == '--':
            val = float('Nan')
        else:
            num_reported_vals = num_reported_vals +1
        facility[fieldList[i]] = val
    if num_reported_vals == 3:
        facility['Reporting'] = "No"
    else:
        facility['Reporting'] = "Yes"    
    facilities.append(facility)
print(len(facilities))
facilities[0]

1363


{'ReportingDate': '11-16-2020',
 'FACILITY NAME': 'Jamieson Nursing Home',
 'COUNTY': 'ALCONA',
 'TYPE': 'SNF',
 'COMPLIANCE': '\xa0',
 'NEW RESIDENT CASES': '0',
 'TOTAL RESIDENT CASES': '0',
 'NEW RESIDENT DEATHS': '0',
 'TOTAL RESIDENT DEATHS': '0',
 'NEW STAFF CASES': '0',
 'TOTAL STAFF CASES': '0',
 'NEW STAFF DEATHS': '0',
 'TOTAL STAFF DEATHS': '0',
 'Reporting': 'Yes'}

In [23]:
fieldList

['FACILITY NAME',
 'COUNTY',
 'TYPE',
 'COMPLIANCE',
 'NEW RESIDENT CASES',
 'TOTAL RESIDENT CASES',
 'NEW RESIDENT DEATHS',
 'TOTAL RESIDENT DEATHS',
 'NEW STAFF CASES',
 'TOTAL STAFF CASES',
 'NEW STAFF DEATHS',
 'TOTAL STAFF DEATHS']

In [24]:
facilities_df = pd.DataFrame(facilities, dtype=np.int)
facilities_df = facilities_df.astype({'ReportingDate': str, 'FACILITY NAME': str, 'COUNTY': str, 'TYPE': str,
                                      'NEW RESIDENT CASES': float, 'TOTAL RESIDENT CASES': float, 'NEW RESIDENT DEATHS': float,
                      'TOTAL RESIDENT DEATHS': float, 'NEW STAFF CASES': float, 'TOTAL STAFF CASES': float,
                      'NEW STAFF DEATHS': float, 'TOTAL STAFF DEATHS': float})

In [25]:
facilities_df.dtypes

ReportingDate             object
FACILITY NAME             object
COUNTY                    object
TYPE                      object
COMPLIANCE                object
NEW RESIDENT CASES       float64
TOTAL RESIDENT CASES     float64
NEW RESIDENT DEATHS      float64
TOTAL RESIDENT DEATHS    float64
NEW STAFF CASES          float64
TOTAL STAFF CASES        float64
NEW STAFF DEATHS         float64
TOTAL STAFF DEATHS       float64
Reporting                 object
dtype: object

In [26]:
facilities_df.sort_values(by=['NEW RESIDENT CASES'], ascending=False).head(10)

,ReportingDate,FACILITY NAME,COUNTY,TYPE,COMPLIANCE,NEW RESIDENT CASES,TOTAL RESIDENT CASES,NEW RESIDENT DEATHS,TOTAL RESIDENT DEATHS,NEW STAFF CASES,TOTAL STAFF CASES,NEW STAFF DEATHS,TOTAL STAFF DEATHS,Reporting
585,11-16-2020,Valley View Care Center,KENT,SNF,,50.0,127.0,4.0,6.0,46.0,82.0,0.0,0.0,Yes
797,11-16-2020,Aspen Grove Assisted Living,MONROE,HFA,,41.0,81.0,7.0,11.0,18.0,34.0,0.0,0.0,Yes
1006,11-16-2020,Villa at West Branch,OGEMAW,SNF,,37.0,48.0,2.0,4.0,0.0,1.0,0.0,0.0,Yes
1007,11-16-2020,Aspirus Ontonagon Hospital LTC,ONTONAGON,SNF,,30.0,59.0,2.0,2.0,12.0,26.0,0.0,0.0,Yes
1101,11-16-2020,"HealthSource Saginaw, Inc",SAGINAW,SNF,,30.0,64.0,2.0,3.0,9.0,80.0,0.0,0.0,Yes
533,11-16-2020,Holland Home Raybrook Manor,KENT,HFA,,28.0,48.0,0.0,1.0,9.0,23.0,0.0,0.0,Yes
845,11-16-2020,SKLD Whitehall,MUSKEGON,SNF,,25.0,56.0,1.0,3.0,15.0,52.0,0.0,0.0,Yes
7,11-16-2020,Ely Manor Nursing & Rehab,ALLEGAN,SNF,,24.0,25.0,0.0,0.0,6.0,9.0,0.0,0.0,Yes
1230,11-16-2020,Aerius Health Center,WAYNE,SNF,,23.0,45.0,0.0,6.0,19.0,32.0,0.0,0.0,Yes
559,11-16-2020,Rest Haven Home,KENT,HFA,,22.0,29.0,1.0,1.0,15.0,18.0,0.0,0.0,Yes


In [27]:
facilities_df.dtypes
facilities_df.convert_dtypes().dtypes

ReportingDate            string
FACILITY NAME            string
COUNTY                   string
TYPE                     string
COMPLIANCE               string
NEW RESIDENT CASES        Int64
TOTAL RESIDENT CASES      Int64
NEW RESIDENT DEATHS       Int64
TOTAL RESIDENT DEATHS     Int64
NEW STAFF CASES           Int64
TOTAL STAFF CASES         Int64
NEW STAFF DEATHS          Int64
TOTAL STAFF DEATHS        Int64
Reporting                string
dtype: object

In [28]:
facilities_df.sort_values(by=['Reporting', 'NEW RESIDENT CASES'], ascending=False).to_csv('Reporting_data/MI_' + reporting_date + 'LTC_data.csv', index=False)

In [29]:
facilites_reporting_cnt = len(facilities_df[facilities_df['Reporting'] == 'Yes'])
facilites_reporting_cnt

1363

In [30]:
facilities_reporting_df = facilities_df[facilities_df['Reporting'] == 'Yes']
facilities_NOT_reporting_df = facilities_df[facilities_df['Reporting'] == 'No']

In [31]:
print(facilities_reporting_df['TYPE'].value_counts())
print(facilities_reporting_df.sum())

AFC    619
SNF    440
HFA    304
Name: TYPE, dtype: int64
ReportingDate            11-16-202011-16-202011-16-202011-16-202011-16-...
FACILITY NAME            Jamieson Nursing HomeLincoln Haven Nursing & R...
COUNTY                   ALCONAALCONAALGERALLEGANALLEGANALLEGANALLEGANA...
TYPE                     SNFSNFSNFSNFAFCHFAAFCSNFAFCAFCSNFAFCAFCHFAAFCA...
COMPLIANCE                     NOT IN COMPLIANCE  NOT IN COMPLIANCE NOT...
NEW RESIDENT CASES                                                     991
TOTAL RESIDENT CASES                                                 12449
NEW RESIDENT DEATHS                                                    166
TOTAL RESIDENT DEATHS                                                 3018
NEW STAFF CASES                                                       1055
TOTAL STAFF CASES                                                     8684
NEW STAFF DEATHS                                                         0
TOTAL STAFF DEATHS                        

In [32]:
facilities_NOT_reporting_df['TYPE'].value_counts()

Series([], Name: TYPE, dtype: int64)

In [33]:
facilities_df.head(10)

,ReportingDate,FACILITY NAME,COUNTY,TYPE,COMPLIANCE,NEW RESIDENT CASES,TOTAL RESIDENT CASES,NEW RESIDENT DEATHS,TOTAL RESIDENT DEATHS,NEW STAFF CASES,TOTAL STAFF CASES,NEW STAFF DEATHS,TOTAL STAFF DEATHS,Reporting
0,11-16-2020,Jamieson Nursing Home,ALCONA,SNF,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Yes
1,11-16-2020,Lincoln Haven Nursing & Rehab. Community,ALCONA,SNF,,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,Yes
2,11-16-2020,Medilodge of Munising,ALGER,SNF,,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Yes
3,11-16-2020,Allegan County Medical Care Community,ALLEGAN,SNF,,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,Yes
4,11-16-2020,Beacon Home at The Oaks,ALLEGAN,AFC,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Yes
5,11-16-2020,Briarwood Assisted Living,ALLEGAN,HFA,,1.0,1.0,0.0,0.0,2.0,3.0,0.0,0.0,Yes
6,11-16-2020,Country Liv - Inn Inc,ALLEGAN,AFC,NOT IN COMPLIANCE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
7,11-16-2020,Ely Manor Nursing & Rehab,ALLEGAN,SNF,,24.0,25.0,0.0,0.0,6.0,9.0,0.0,0.0,Yes
8,11-16-2020,Golden Orchards I,ALLEGAN,AFC,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Yes
9,11-16-2020,Golden Orchards II,ALLEGAN,AFC,NOT IN COMPLIANCE,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,Yes
